In [1]:
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.merge import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


In [2]:
# define inpur sequence
in_seq1 = np.arange(10, 100, 10)
in_seq2 = np.arange(15, 105, 10)
out_seq = in_seq1 + in_seq2
#
in_seq1_res = in_seq1.reshape(in_seq1.shape[0], 1)
in_seq2_res = in_seq2.reshape(in_seq2.shape[0], 1)
out_seq_res = out_seq.reshape(out_seq.shape[0], 1)
dataset = np.hstack((in_seq1_res, in_seq2_res, out_seq_res))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [3]:
n_step = 3
pred_step = 1
def split_sequence(series, n_step, pred_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-(n_step+pred_step-2)):
        x_list.append(series[s:s+n_step, :2]),
        y_list.append(series[s+n_step-1, -1])
    X, y = np.array(x_list), np.array(y_list)
    return X, y
#
X, y = split_sequence(dataset, n_step, pred_step)
for i in range(X.shape[0]):
    print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [4]:
print(X.shape, y.shape)

(7, 3, 2) (7,)


In [5]:
n_features = X.shape[2]

In [6]:
# define model
model = Sequential()
model.add(Conv1D(64, 2, activation='relu' , input_shape=(n_step, n_features)))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam' , loss='mse' )

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2, 64)             320       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 3,621
Trainable params: 3,621
Non-trainable params: 0
_________________________________________________________________


- Param # 320 = 64 * 2 * (2 feature) + 64 (bias) = 320

In [8]:
X_cnn = X.reshape(X.shape[0], X.shape[1], n_features)
print(X.shape, X_cnn.shape)

(7, 3, 2) (7, 3, 2)


There is no need to reshape X

model.fit(X, y, epochs=1000, verbose=0)

In [10]:
x_test = np.array([[70, 75], [80, 85], [90, 95]])
print(x_test.shape)
x_test_cnn = x_test.reshape(1, x_test.shape[0], x_test.shape[1])
yhat = model.predict(x_test_cnn)
print(yhat)

(3, 2)
[[187.4296]]


In [12]:
## visualize model
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

### the model
<img src="./model_plot.png" alt="Drawing" style="width: 800px;"/>